In [10]:
!pip install gdown
import gdown
from pathlib import Path
from dataclasses import dataclass
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import read_yaml, create_directories, get_size
from fraud_detection_project import logger

ModuleNotFoundError: No module named 'fraud_detection_project'

In [ ]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Ensure artifacts root exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        local_data_file = Path(config.local_data_file)
        local_data_file.parent.mkdir(parents=True, exist_ok=True)

        return DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=local_data_file
        )


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self, gdrive_file_id: str = None):
        """
        Downloads the dataset if it doesn't exist.
        If `gdrive_file_id` is provided, downloads from Google Drive,
        otherwise downloads from the URL in the config.
        """
        if self.config.local_data_file.exists():
            logger.info(f"File already exists: {get_size(self.config.local_data_file)}")
            return

        if gdrive_file_id:
            url = f'https://drive.google.com/uc?id={gdrive_file_id}'
            gdown.download(url, str(self.config.local_data_file), quiet=False)
            logger.info(f"Downloaded {self.config.local_data_file} from Google Drive.")
        else:
            import urllib.request as request
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with headers: {headers}")

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)

    # Provide Google Drive file ID if you want to download from Drive
    gdrive_file_id = '1JRNRK-Iv5jD3aqf9F6Pn-eirYZreGbsi'
    data_ingestion.download_file(gdrive_file_id=gdrive_file_id)

except Exception as e:
    raise e